In [1]:
import tensorflow as tf

import numpy as np
import cupy as cp
import scipy
import scipy.misc  # for imresize
# import tdb
import matplotlib.pyplot as plt
%matplotlib inline

import time
from PIL import Image, ImageOps
import os # for directory listings
import pickle



# We can import pre trained data files fro google tensorflow using slim
slim = tf.contrib.slim

from nets import inception
from preprocessing import inception_preprocessing

image_size = inception.inception_v1.default_image_size

IMAGE_W=224
image_size


/home/n/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


224

In [2]:

import os
import time
from PIL import Image, ImageOps
import numpy as np
import scipy.misc
from six.moves import urllib

def get_resized_image(img_path, width, height, save=True):
    image = Image.open(img_path)
    # PIL is column major so you have to swap the places of width and height
    image = ImageOps.fit(image, (width, height), Image.ANTIALIAS)
    if save:
        image_dirs = img_path.split('/')
        image_dirs[-1] = 'resized_' + image_dirs[-1]
        out_path = '/'.join(image_dirs)
        if not os.path.exists(out_path):
            image.save(out_path)
    image = np.asarray(image, np.float32)
    return np.expand_dims(image, 0)

def generate_noise_image(content_image, width, height, noise_ratio=0.6):
    noise_image = np.random.uniform(-20, 20, (1, height, width, 3)).astype(np.float32)
    return noise_image * noise_ratio + content_image * (1 - noise_ratio)

def save_image(path, image):
    image = image[0]
    image = np.clip(image, 0, 255).astype('uint8')
    scipy.misc.imsave(path, image)

def safe_mkdir(path):
    """ Create a directory if there isn't one already. """
    try:
        os.mkdir(path)
    except OSError:
        pass

In [3]:
content_img = '/home/n/neural-style/images/makise.jpg'
style_img = '/home/n/smooth_ride.jpg'
img_width, img_height = 224,224

In [4]:
content_img = get_resized_image(content_img, img_width, img_height)
style_img = get_resized_image(style_img, img_width, img_height)
initial_img = generate_noise_image(content_img, img_width, img_height)

        ###############################
        ## TO DO
        ## create global step (gstep) and hyperparameters for the model
content_layer = 'Mixed_4b'
style_layers = ['Conv2d_1a_7x7', 'Conv2d_2c_3x3','Mixed_3b','Mixed_4d']
content_w = 0.01
style_w = 29
style_layer_w = [0.5, 1.0, 1.5, 3.0] 
gstep = tf.Variable(0, dtype=tf.int32, 
                                trainable=False, name='global_step')
lr = 0.01
        ###############################
mean_pixels = np.array([123.68, 116.779, 103.939]).reshape((1,1,1,3))
        
content_img -= mean_pixels
style_img -= mean_pixels

In [5]:
def content_loss_func(sess, model):
    """
    Content loss function as defined in the paper.
    """
    def _content_loss(p, x):
        # N is the number of filters (at layer l).
        N = p.shape[3]
        # M is the height times the width of the feature map (at layer l).
        M = p.shape[1] * p.shape[2]
        # Interestingly, the paper uses this form instead:
        #
        #   0.5 * tf.reduce_sum(tf.pow(x - p, 2)) 
        #
        # But this form is very slow in "painting" and thus could be missing
        # out some constants (from what I see in other source code), so I'll
        # replicate the same normalization constant as used in style loss.
        return (1 / (4 * N * M)) * tf.reduce_sum(tf.pow(x - p, 2))
    return _content_loss(sess.run(model['Mixed_4b']), model['Mixed_4b'])

In [6]:
def style_loss_func(sess, model):
    """
    Style loss function as defined in the paper.
    """
    def _gram_matrix(F, N, M):
        """
        The gram matrix G.
        """
        Ft = tf.reshape(F, (M, N))
        return tf.matmul(tf.transpose(Ft), Ft)

    def _style_loss(a, x):
        """
        The style loss calculation.
        """
        # N is the number of filters (at layer l).
        N = a.shape[3]
        # M is the height times the width of the feature map (at layer l).
        M = a.shape[1] * a.shape[2]
        # A is the style representation of the original image (at layer l).
        A = _gram_matrix(a, N, M)
        # G is the style representation of the generated image (at layer l).
        G = _gram_matrix(x, N, M)
        result = (1 / (4 * N**2 * M**2)) * tf.reduce_sum(tf.pow(G - A, 2))
        return result

    E = [_style_loss(sess.run(model[layer_name]), model[layer_name]) for layer_name in style_layers]
    W = [_ for _ in style_layer_w]
    loss = sum([W[l] * E[l] for l in range(len(style_layers))])
    return loss

In [7]:
with tf.variable_scope('input') as scope:
    input_img = tf.get_variable('in_img',shape=([1, img_height, img_width, 3]),dtype=tf.float32,initializer=tf.zeros_initializer())
            

In [8]:
with slim.arg_scope(inception.inception_v1_arg_scope()):
    _, end_points = inception.inception_v1(input_img, num_classes=1001, is_training=False)

        # Create an operation that loads the pre-trained model from the checkpoint
init_fn = slim.assign_from_checkpoint_fn(
    os.path.join('home/n/models/inception_/', '/home/n/data/inception_v1.ckpt'),
    slim.get_model_variables('InceptionV1')
        )

In [9]:
sess = tf.InteractiveSession()

In [10]:
init_fn(sess)

INFO:tensorflow:Restoring parameters from /home/n/data/inception_v1.ckpt


In [11]:
sess.run(tf.global_variables_initializer())

In [12]:
sess.run(input_img.assign(content_img)) 
content_loss = content_loss_func(sess, end_points)

In [13]:
sess.run(input_img.assign(style_img)) 
style_loss = style_loss_func(sess, end_points)

In [14]:
total_loss = 0.01 * content_loss + 1 * style_loss

In [15]:
sess.run(tf.global_variables_initializer())
sess.run(input_img.assign(initial_img)) 

array([[[[83.26459 , 48.41996 , 64.1406  ],
         [84.435104, 55.179073, 66.918045],
         [76.27604 , 45.561974, 46.972034],
         ...,
         [63.091656, 58.353867, 54.650482],
         [64.78653 , 58.952614, 59.64515 ],
         [56.044834, 59.267975, 61.521233]],

        [[88.59465 , 45.715256, 72.36557 ],
         [68.05325 , 58.222874, 55.74738 ],
         [68.9247  , 54.034176, 28.75959 ],
         ...,
         [58.31027 , 61.1719  , 68.10062 ],
         [62.775604, 75.18086 , 52.066574],
         [53.945282, 53.392014, 45.397465]],

        [[84.904335, 59.21613 , 74.99996 ],
         [79.5144  , 50.60715 , 70.30597 ],
         [54.041676, 47.315678, 28.30484 ],
         ...,
         [52.96541 , 68.57007 , 51.22074 ],
         [48.945904, 67.336975, 53.39328 ],
         [56.94664 , 70.9372  , 59.171776]],

        ...,

        [[41.99567 , 48.135147, 20.265556],
         [25.67918 , 38.220512, 22.33934 ],
         [24.836193, 23.800167, 28.830517],
         ...,


In [16]:
optimizer = tf.train.AdamOptimizer(0.001).minimize(total_loss)

def save_image(path, image):
    # Output should add back the mean.
    image = image + mean_pixels
    # Get rid of the first useless dimension, what remains is the image.
    image = image[0]
    image = np.clip(image, 0, 255).astype('uint8')
    scipy.misc.imsave(path, image)

In [17]:
ITERATIONS = 30
OUTPUT_DIR = 'output/'

In [18]:
sess.run(tf.global_variables_initializer())
init_fn(sess)
sess.run(input_img.assign(initial_img)) 
for it in range(ITERATIONS):
    
    initial_img,_ = sess.run([input_img,optimizer])
    if it%10 == 0:
        # Print every 100 iteration.
#         mixed_image = sess.run(end_points)
        print('Iteration %d' % (it))
        
#         if not os.path.exists(OUTPUT_DIR):
#             os.mkdir(OUTPUT_DIR)
#         print(mixed_image)
        filename = 'output/%d.png' % (it)
        save_image(filename, initial_img)


INFO:tensorflow:Restoring parameters from /home/n/data/inception_v1.ckpt
Iteration 0


/home/n/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  if __name__ == '__main__':


Iteration 10
Iteration 20


In [19]:
initial_img.shape

(1, 224, 224, 3)

In [20]:
initial_img

array([[[[83.270546, 48.4206  , 64.12677 ],
         [84.416145, 55.158474, 66.90519 ],
         [76.26795 , 45.546913, 46.96116 ],
         ...,
         [63.080013, 58.343548, 54.64077 ],
         [64.779305, 58.944412, 59.636955],
         [56.04826 , 59.270416, 61.526848]],

        [[88.59198 , 45.711163, 72.35356 ],
         [68.03229 , 58.205685, 55.732002],
         [68.923965, 54.034504, 28.753897],
         ...,
         [58.30015 , 61.162872, 68.09262 ],
         [62.76603 , 75.17094 , 52.05712 ],
         [53.92672 , 53.37088 , 45.398445]],

        [[84.890366, 59.206497, 74.99128 ],
         [79.50449 , 50.597614, 70.29779 ],
         [54.045853, 47.308235, 28.31254 ],
         ...,
         [52.95525 , 68.55714 , 51.213463],
         [48.93887 , 67.32774 , 53.386395],
         [56.939064, 70.92959 , 59.162033]],

        ...,

        [[41.986355, 48.126667, 20.256847],
         [25.683102, 38.22149 , 22.343626],
         [24.831766, 23.80073 , 28.843609],
         ...,


In [22]:
a  = np.arange(10,1000)

In [24]:
b = np.arange(10,100)

In [25]:
b

array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43,
       44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60,
       61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
       78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94,
       95, 96, 97, 98, 99])

In [28]:
b[::3]

array([10, 13, 16, 19, 22, 25, 28, 31, 34, 37, 40, 43, 46, 49, 52, 55, 58,
       61, 64, 67, 70, 73, 76, 79, 82, 85, 88, 91, 94, 97])

In [33]:
b

array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43,
       44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60,
       61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
       78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94,
       95, 96, 97, 98, 99])

In [34]:
np.setdiff1d(a,b)

array([100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
       113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
       126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138,
       139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151,
       152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164,
       165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177,
       178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190,
       191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203,
       204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216,
       217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229,
       230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242,
       243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255,
       256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268,
       269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 28

In [45]:
c = np.append(a,b)

In [39]:
np.bincount(b)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [40]:
b

array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43,
       44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60,
       61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
       78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94,
       95, 96, 97, 98, 99])

In [41]:
import collections

In [46]:
collections.Counter(c)

Counter({10: 2,
         11: 2,
         12: 2,
         13: 2,
         14: 2,
         15: 2,
         16: 2,
         17: 2,
         18: 2,
         19: 2,
         20: 2,
         21: 2,
         22: 2,
         23: 2,
         24: 2,
         25: 2,
         26: 2,
         27: 2,
         28: 2,
         29: 2,
         30: 2,
         31: 2,
         32: 2,
         33: 2,
         34: 2,
         35: 2,
         36: 2,
         37: 2,
         38: 2,
         39: 2,
         40: 2,
         41: 2,
         42: 2,
         43: 2,
         44: 2,
         45: 2,
         46: 2,
         47: 2,
         48: 2,
         49: 2,
         50: 2,
         51: 2,
         52: 2,
         53: 2,
         54: 2,
         55: 2,
         56: 2,
         57: 2,
         58: 2,
         59: 2,
         60: 2,
         61: 2,
         62: 2,
         63: 2,
         64: 2,
         65: 2,
         66: 2,
         67: 2,
         68: 2,
         69: 2,
         70: 2,
         71: 2,
        